In [1]:
import collections
import os
import tensorflow as tf
import pandas as pd

tf.__version__


'2.13.0'

In [2]:
main_dir = r"F:\linux_ubuntu_codes\codes\nlp_lecture"

In [3]:
train_dir = os.path.join(main_dir + r"\train.csv")
test_dir = os.path.join(main_dir + r"\test.csv")

'https://raw.githubusercontent.com/sedeba19/NLP--Random-Models/main/train.csv'

In [4]:
train_dir

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Make a train_df
train_df = pd.read_csv(train_dir)
test_df = pd.read_csv(test_dir)
train_df.head()

,id,keyword,location,text,target
3228,4632,emergency%20services,"Sydney, New South Wales",Goulburn man Henry Van Bilsen missing: Emergen...,1
3706,5271,fear,NaN,The things we fear most in organizations--fluc...,0
6957,9982,tsunami,Land Of The Kings,@tsunami_esh ?? hey Esh,0
2887,4149,drown,NaN,@POTUS you until you drown by water entering t...,0
7464,10680,wounds,"cody, austin follows ?*?",Crawling in my skin\nThese wounds they will no...,1


In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac = 1, random_state = 1)
train_df_shuffled.head()

(7613, 5)

In [7]:
train_df_shuffled.shape

target
0    4342
1    3271
Name: count, dtype: int64

In [8]:
# How many examples of each class?
train_df['target'].value_counts()

(7613, 3263)

In [9]:
# How many total samples?
len(train_df), len(test_df)

Target:1 (real disaster)
Text: 
Beat:B2 MOTOR VEHICLE COLLISION at N 35 ST / FREMONT AV N reported on 8/5/2015 6:52 PM Call# 15000270364
-----

Target:1 (real disaster)
Text: 
Tarp is protecting outfield and cannot be moved. Infield getting deluged.
-----

Target:0 (not a real disaster)
Text: 
A blizzard would be clutch asf ??
-----

Target:0 (not a real disaster)
Text: 
When you're 5 hours late for curfew and have to pray your dog doesn't bark when unlocking the door
-----

Target:0 (not a real disaster)
Text: 
@smoak_queen 'I'm going to be in so much trouble.'
-----


In [ ]:
# Visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index: random_index +5].itertuples():
    _, text, target = row
    if target > 0:
        print(f"Target:{target}", "(real disaster)")
    else:
        print(f"Target:{target}", "(not a real disaster)")
    
    print(f"Text: \n{text}")
    print("-----\n")

### Split data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df['text'],
                                                                            train_df['target'],
                                                                            test_size = 0.1,
                                                                            random_state = 42)

(7613,)

In [12]:
train_df['text'].shape

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [13]:
train_df["text"]

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [14]:
train_df['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [15]:
train_df['text'].to_numpy()

(6851, 762, 6851, 762)

In [ ]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

### Converting text into numbers: Text Vectorization also known as Tokenization

In [17]:
# Get the total words
total_words = 0
for i in train_sentences:
    total_words += len(i.split())

total_words

(6851,)

In [18]:
train_sentences.shape

15

In [19]:
# Get the average words per sentence or line
avg_words_per_sentence = round(total_words/len(train_sentences))
avg_words_per_sentence

In [20]:
# Normally, total number of words is equal or less than the max vocabulary length
max_vocab_length = 10000

# max length of a statement is equal or more than the average word per sentence or line
max_length = 15

In [21]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

# Fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

There's a flood in my street! Sentence length is 29.


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[282,   3, 206,   4,  13, 674,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [22]:
# Create a sample sentence and tokenize it
samp_sentence = "There's a flood in my street!"
print(samp_sentence, f"Sentence length is {len(samp_sentence)}.")
text_vectorizer([samp_sentence])
#len(tf.squeeze(text_vectorizer([samp_sentence])))

Original text: 

 like why on earth would you want anybody to be unhappy don't purposely ruin somebody else's happiness 

Vectorized text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  25,   92,   11, 1081,   80,   12,  145, 3171,    5,   22, 7150,
          64, 9454,  355, 1606]], dtype=int64)>

In [23]:
# Choose random sentence from the training dataset and tokenize it
rand_train_sentence = random.choice(train_sentences)
print(f"Original text: \n\n {rand_train_sentence} \n\nVectorized text:")
text_vectorizer([rand_train_sentence])

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'are',
 'be',
 'was',
 'have',
 'like',
 'as',
 'me',
 'but',
 'up',
 'just',
 'so',
 'im',
 'not',
 'amp',
 'your',
 'out',
 'all',
 'after',
 'its',
 'has',
 'no',
 'will',
 'an',
 'fire',
 'when',
 'if',
 'we',
 'get',
 'now',
 'more',
 'via',
 'new',
 'about',
 'what',
 'people',
 'or',
 'news',
 'he',
 'they',
 'over',
 'one',
 'been',
 'how',
 'dont',
 'who',
 'video',
 'into',
 'were',
 'do',
 'us',
 'can',
 'emergency',
 '2',
 'disaster',
 'there',
 'her',
 'some',
 'than',
 'still',
 'would',
 'his',
 'police',
 'crash',
 'burning',
 'suicide',
 'body',
 'california',
 'back',
 'time',
 'buildings',
 'had',
 'why',
 'off',
 'them',
 'got',
 'man',
 'see',
 'storm',
 'know',
 'going',
 'cant',
 'world',
 'first',
 'day',
 'rt',
 'nuclear',
 'love',
 'youtube',
 'our',
 'attack',
 'go',
 'fires',
 'two',
 'their',
 'bomb',

In [24]:
# Get unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab

10000

In [25]:
len(words_in_vocab)

['', '[UNK]', 'the', 'a', 'in']

In [26]:
# Top 5 words in words_in_vocab
words_in_vocab[:5]

['pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']

In [ ]:
# Least 5 words in words_in_vocab
words_in_vocab[-5:]

### Embedding layer

To make our embedding, we are going to use TensorFlow’s embedding layer: https://www.tensorflow.org/api/docs/python/tf/keras/layers/

The parameters are below mostly used.
* input_dim = size of the vocabulary
* output_dim = size of the output embedding vector.  For example, a value of 100 would mean each token gets represented by a vector 100 long
* input_length = length of the sequences being passed to the embedding layer

In [28]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 64,
                             embeddings_initializer = "uniform",
                             input_length = max_length)

Original text: Train derailment: In Patna no news of any casualty so far - The Indian Express http://t.co/YH5VETm0YZ http://t.co/17Wgug8z0M

Vectorized text: [[ 118  364    4 4920   41   57    6  234  634   31  601    2 1286 1536
     1]]

Vectorized text shape: (1, 15)
Embbedded text: [[[-4.43637632e-02 -1.26107335e-02  2.84378603e-03 -3.35236788e-02
   -3.13915834e-02  4.43817414e-02  4.85944487e-02  1.71508528e-02
    4.01938818e-02  3.83564867e-02 -1.96816921e-02 -4.18531895e-03
   -4.44837324e-02  4.33234312e-02 -2.50958093e-02 -4.40843217e-02
   -1.44094713e-02  3.44209112e-02  6.63088635e-03  3.54768522e-02
    2.20246427e-02  4.97455485e-02  4.45870794e-02  4.02387418e-02
    2.00218596e-02  4.81777675e-02  3.22533138e-02  3.16465534e-02
   -2.65120268e-02  3.31873037e-02 -4.69052792e-02 -9.49148089e-03
    5.29532507e-03 -7.48661906e-03 -8.39617103e-03 -3.50897908e-02
    1.04417205e-02  2.90989988e-02  4.62394953e-03  3.83883826e-02
    3.35663557e-03  1.31147243e-02  1.79450

In [ ]:
# Get a random sentence from the training set and return Original text, Vectorized text, Embedded text
rand_sentence = random.choice(train_sentences)
print(f"Original text: {rand_sentence}\n")
print(f"Vectorized text: {text_vectorizer([rand_sentence])}\n")
print(f"Vectorized text shape: {text_vectorizer([rand_sentence]).shape}")
print(f"Embbedded text: {embedding(text_vectorizer([rand_sentence]))}")
print(f"Embedded text shape: {embedding(text_vectorizer([rand_sentence])).shape}")

### Model_6: TensorFlow Hub Pretrained Feature Extractor: Universal Sentence Encoder

https://tfhub.dev/google/universal-sentence-encoder/4
See how the USE was created here: https://arvix.org/abs/1803.11175

In [30]:
# Create a Keras Layer using the USE Pretrained Layer from TensorFlow Hub
import tensorflow_hub as hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape = [],
                                        dtype = tf.string,
                                        trainable = False,
                                        name = 'USE')

In [31]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation = 'relu'),
    layers.Dense(1, activation = 'sigmoid')
],  name = "Model_6_USE")

# Compile the model
model_6.compile(loss = 'binary_crossentropy',
                optimizer= tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

Model: "Model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [32]:
model_6.summary()

24/24 [==============================] - 0s 3ms/step


array([[0.50619537],
       [0.49603048],
       [0.5036392 ],
       [0.51526636],
       [0.5236116 ],
       [0.5050182 ],
       [0.495656  ],
       [0.50734544],
       [0.5103289 ],
       [0.47278407],
       [0.5112029 ],
       [0.47847378],
       [0.4973488 ],
       [0.51539165],
       [0.48423472],
       [0.49757752],
       [0.48505464],
       [0.5233879 ],
       [0.4892447 ],
       [0.48812613],
       [0.4896002 ],
       [0.4796798 ],
       [0.5146651 ],
       [0.50801414],
       [0.5103214 ],
       [0.5094193 ],
       [0.5277337 ],
       [0.49066928],
       [0.4969799 ],
       [0.523704  ],
       [0.4987458 ],
       [0.52059746],
       [0.5038079 ],
       [0.5195947 ],
       [0.50522876],
       [0.50232804],
       [0.49221134],
       [0.4825051 ],
       [0.5068269 ],
       [0.50641847],
       [0.48926136],
       [0.5140351 ],
       [0.494423  ],
       [0.48895985],
       [0.49635008],
       [0.5157402 ],
       [0.49329177],
       [0.507

In [33]:
# Make predictions with USE TF Hub Model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs

(762,)

In [34]:
val_labels.shape

TensorShape([762])

In [35]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds.shape

In [36]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_6_accuracy_score = accuracy_score(val_labels, model_6_preds)
model_6_precision, model_6_recall, model_6_f1score, _, =precision_recall_fscore_support(val_labels, model_6_preds, average = "weighted")

{'model_6_accuracy_score': 0.5328083989501312,
 'model_6_precision': 0.5646698661818376,
 'model_6_recall': {0.5328083989501312},
 'model_6_f1score': {0.5237200785651015}}

In [ ]:
dict_result = {"model_6_accuracy_score": model_6_accuracy_score,
               "model_6_precision": model_6_precision,
               "model_6_recall": {model_6_recall},
               "model_6_f1score": {model_6_f1score}}
dict_result